In [4]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

# 데이터를 pandas dataframe으로 불러온다
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# print(train.head())


In [5]:
# 크게 관련 없는 columns를 삭제한다
# Ticket, Fare, Cabin, PasengerId
train = train_df.drop(["Ticket", "Fare", "Cabin", "PassengerId", "Name"], axis=1, )
test = test_df.drop(["Ticket", "Fare", "Cabin", "PassengerId", "Name"], axis=1, )

In [ ]:
# 각 Data에 들어 있는 

In [ ]:
# NaN값이 들어있는 columns를 확인한다
train.isnull().sum(axis=0)

* Age값에 Nan값이 꽤 많이 들어 있는 것으로 보인다
* 일단은 Train Data의 평균을 이용해서 NaN 값을 채우는 것이 좋아보인다
* 데이터의 통일성을 위해 Test Data 또한 Train Data의 Mean 값을 이용해 보간한다.

In [ ]:
train["Age"].fillna(train["Age"].mean(), inplace=True)
test["Age"].fillna(train["Age"].mean(), inplace=True)


In [ ]:
# 각 column별로 값을 확인한다
for index in train.columns:
    print(train[index].value_counts())
    print()

In [ ]:
# 넘파이배열로 변환
train = train.values
test = test.values
set = [train,test]

In [ ]:
# male, female 값을 각각 0과 1로 인코딩한다
sex_i = 3
embark_i = 7
for data in set:
    sex_i -= 1
    embark_i -= 1
    for i in range(data.shape[0]):
        if data[i,sex_i] == 'male' :
            data[i,sex_i] = 0
        else :
            data[i,sex_i] = 1
        if data[i,embark_i] == 'S':
            data[i,embark_i] = 0
        elif data[i,embark_i] == 'Q':
            data[i,embark_i] = 1
        else :
            data[i,embark_i] = 2

In [ ]:
# 데이터 형변환
train = train.astype(np.float64)
test = test.astype(np.float64)

한 sample은 총 6개의 feature를 가지고 있다.

3개의 Fully Connected Layer를 이용한 신경망을 이용해보자


In [ ]:
# 훈련용 변수와 정답 레이블을 분리한다
x_train = train[:,1:]
y_train = train[:,0]

In [ ]:
# keras모델 설계
model = models.Sequential()
model.add(layers.Dense(16, activation="relu", input_shape=(6,)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["binary_accuracy"])



In [ ]:
history = model.fit(
    x_train, y_train, epochs=200, batch_size=20, validation_split=0.2, shuffle = True, verbose=1
)

history_dict = history.history
history_dict.keys()

In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# ‘b’는 파란색 실선을 의미합니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # 그래프를 초기화합니다
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [28]:
result = model.predict(test)
result = result.reshape(418)
# result.shape



(418,)

In [30]:
my_submission = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': result})
my_submission.to_csv('submission.csv', index=False)